In [1]:
import os
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import json
import pinecone
import dotenv

dotenv.load_dotenv()



True

In [10]:
from pinecone import Pinecone, ServerlessSpec

In [3]:
# Set up API keys from .env file
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

# Initialize clients
client = OpenAI(api_key=OPENAI_API_KEY)
pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)

# Index configuration
INDEX_NAME = "simple-rag-index"
DIMENSION = 1536  # OpenAI embeddings dimension

print("Clients initialized successfully!")

Clients initialized successfully!


In [11]:
# Create or connect to Pinecone index
try:
    # Try to create index if it doesn't exist
    if INDEX_NAME not in [index.name for index in pc.list_indexes()]:
        pc.create_index(
            name=INDEX_NAME,
            dimension=DIMENSION,
            metric="cosine", 
            spec=ServerlessSpec(
                    cloud="aws",
                    region="us-east-1")
        )
        print(f"Created new index: {INDEX_NAME}")
    else:
        print(f"Index {INDEX_NAME} already exists")
    
    # Connect to the index
    index = pc.Index(INDEX_NAME)
    print(f"Connected to index: {INDEX_NAME}")
    
except Exception as e:
    print(f"Error with index: {e}")

Created new index: simple-rag-index
Connected to index: simple-rag-index


e:\Gen AI\RAG_Projects\naive_rag\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# Sample documents (replace with your own documents)
documents = [
    "Artificial Intelligence is transforming the way we work and live. It includes machine learning, deep learning, and natural language processing.",
    "Machine learning is a subset of AI that enables computers to learn and improve from experience without being explicitly programmed.",
    "Deep learning uses neural networks with multiple layers to model and understand complex patterns in data.",
    "Natural language processing (NLP) helps computers understand, interpret and generate human language in a valuable way.",
    "Python is a popular programming language for AI and machine learning due to its simplicity and powerful libraries.",
    "Data science involves extracting insights from structured and unstructured data using scientific methods and algorithms."
]

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

# Split documents into chunks
chunks = []
for i, doc in enumerate(documents):
    doc_chunks = text_splitter.split_text(doc)
    for j, chunk in enumerate(doc_chunks):
        chunks.append({
            "id": f"doc_{i}_chunk_{j}",
            "text": chunk,
            "source": f"document_{i}"
        })

print(f"Created {len(chunks)} chunks from {len(documents)} documents")

Created 6 chunks from 6 documents


In [13]:
# Create embeddings and store in Pinecone
embeddings_model = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

# Process chunks and store in Pinecone
vectors_to_upsert = []

for chunk in chunks:
    # Create embedding for the chunk
    embedding = embeddings_model.embed_query(chunk["text"])
    
    # Prepare vector for upsert
    vector = {
        "id": chunk["id"],
        "values": embedding,
        "metadata": {
            "text": chunk["text"],
            "source": chunk["source"]
        }
    }
    vectors_to_upsert.append(vector)

# Upsert vectors to Pinecone
try:
    index.upsert(vectors=vectors_to_upsert)
    print(f"Successfully stored {len(vectors_to_upsert)} vectors in Pinecone")
except Exception as e:
    print(f"Error storing vectors: {e}")

Successfully stored 6 vectors in Pinecone


In [14]:
def simple_rag_query(question, top_k=3):
    """
    Simple RAG function that retrieves relevant documents and generates an answer
    """
    # Create embedding for the question
    question_embedding = embeddings_model.embed_query(question)
    
    # Search for similar vectors in Pinecone
    results = index.query(
        vector=question_embedding,
        top_k=top_k,
        include_metadata=True
    )
    
    # Extract relevant context
    context = ""
    sources = []
    for match in results.matches:
        context += match.metadata["text"] + "\n\n"
        sources.append(match.metadata["source"])
    
    # Generate response using OpenAI
    prompt = f"""
    Context: {context}
    
    Question: {question}
    
    Please answer the question based on the provided context. If the context doesn't contain enough information to answer the question, say so.
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=500
        )
        
        answer = response.choices[0].message.content
        
        return {
            "question": question,
            "answer": answer,
            "sources": list(set(sources)),
            "context_snippets": [match.metadata["text"] for match in results.matches]
        }
    
    except Exception as e:
        return {"error": f"Error generating response: {e}"}

print("RAG query function created successfully!")

RAG query function created successfully!


In [15]:
# Test the RAG system with example queries
test_questions = [
    "What is machine learning?",
    "How does deep learning work?",
    "What programming language is good for AI?",
    "What is data science?"
]

print("Testing RAG System:")
print("=" * 50)

for question in test_questions:
    print(f"\nQuestion: {question}")
    result = simple_rag_query(question)
    
    if "error" in result:
        print(f"Error: {result['error']}")
    else:
        print(f"Answer: {result['answer']}")
        print(f"Sources: {', '.join(result['sources'])}")
    print("-" * 30)

Testing RAG System:

Question: What is machine learning?
Answer: Machine learning is a subset of AI that enables computers to learn and improve from experience without being explicitly programmed.
Sources: document_1, document_2, document_0
------------------------------

Question: How does deep learning work?
Answer: Deep learning works by using neural networks with multiple layers to model and understand complex patterns in data. These networks are trained on large datasets to recognize patterns and make predictions or decisions based on the input data.
Sources: document_1, document_2, document_0
------------------------------

Question: What programming language is good for AI?
Answer: Python is a good programming language for AI, especially for machine learning and deep learning, due to its simplicity and powerful libraries. It is widely used in the field of artificial intelligence.
Sources: document_0, document_1, document_4
------------------------------

Question: What is data s